<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Anton_Shalin/Resize_Distribute_Dataset_for_YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import random
from PIL import Image
import cv2
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')
!rm -r sample_data

Mounted at /content/drive


In [ ]:
# Функция для удаления директории
def delete_directory(path: str):
    if os.path.exists(path):
        shutil.rmtree(path)

# Функция для удаления файла
def delete_file(file_name: str):
  if os.path.exists(file_name):
    os.remove(file_name)

# Функция для чтения списка классов из файла
def read_classes(file_name: str):
  with open(file_name, 'r', encoding='utf-8') as file:
    classes = [line.strip() for line in file if line.strip()]
  return classes

# Функция для нахождения пар изображение-метка
def find_image_label_pairs(src_dir):
    images = {}
    labels = {}

    # Проходим по всем файлам во всех поддиректориях
    for root, _, files in os.walk(src_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
                base_name = os.path.splitext(file)[0]
                images[base_name] = os.path.join(root, file)
            elif file.lower().endswith('.txt'):
                base_name = os.path.splitext(file)[0]
                labels[base_name] = os.path.join(root, file)

    # Сопоставляем изображения и метки
    pairs = []
    for base_name, img_path in images.items():
        txt_path = labels.get(base_name)
        if txt_path:
            pairs.append((img_path, txt_path))

    return pairs

# Функция для изменения размера и сохранения изображения
def resize_and_save_image(src_img_path, dst_img_path, img_size, keep_proportions=True):
    original_image = Image.open(src_img_path)
    original_width, original_height = original_image.size

    target_image = Image.new('RGB', (img_size, img_size), color='black')

    changed_width = -1
    changed_height = -1
    offset_x = 0
    offset_y = 0

    # Если обе стороны изображения меньше целевого размера, накладываем на черный фон
    if original_width <= img_size and original_height <= img_size:
        offset_x = (img_size - original_width) // 2
        offset_y = (img_size - original_height) // 2
        target_image.paste(original_image, (offset_x, offset_y))
    else:
        # Меняем размер изображения с учетом пропорций или без
        if keep_proportions:
            original_image.thumbnail((img_size, img_size), Image.Resampling.LANCZOS)
            changed_width = original_image.width
            changed_height = original_image.height
            offset_x = (img_size - changed_width) // 2
            offset_y = (img_size - changed_height) // 2
            target_image.paste(original_image, (offset_x, offset_y))
        else:
            resized_image = original_image.resize((img_size, img_size))
            target_image.paste(resized_image, (0, 0))
            changed_width = img_size
            changed_height = img_size

    target_image.save(dst_img_path, format='JPEG')

    return original_width, original_height, changed_width, changed_height, offset_x, offset_y

# Функция для изменения разметки YOLO в соответствии с изменением размера изображения
def change_yolo_markup(input_txt_path, original_width, original_height, changed_width, changed_height, offset_x, offset_y, img_size, output_txt_path):
    with open(input_txt_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    is_bbox_valid = False
    with open(output_txt_path, 'w', encoding='utf-8') as file:
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # Адаптация размеров и позиций меток
            if changed_width != -1:
                x_center *= changed_width
                y_center *= changed_height
                width *= changed_width
                height *= changed_height
            else:
                x_center *= original_width
                y_center *= original_height
                width *= original_width
                height *= original_height

            # Проверка минимального размера метки
            if width >= 3 and height >= 3:
                is_bbox_valid = True
                x_center = (x_center + offset_x) / img_size
                y_center = (y_center + offset_y) / img_size
                width /= img_size
                height /= img_size
                file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    return is_bbox_valid

# Функция для нормализации и копирования пары изображение-метка
def pair_normalize_copy(img_fname1, txt_fname1, img_fname2, txt_fname2, img_size):
    original_width, original_height, changed_width, changed_height, offset_x, offset_y = resize_and_save_image(img_fname1, img_fname2, img_size, keep_proportions)

    # Проверка, если изображение не изменилось после ресайза
    if original_width == img_size and original_width == changed_width and original_height == changed_height:
        shutil.copyfile(txt_fname1, txt_fname2)
        return

    # Изменение разметки YOLO
    is_bbox_valid = change_yolo_markup(txt_fname1, original_width, original_height, changed_width, changed_height, offset_x, offset_y, img_size, txt_fname2)

    # Если после ресайза не осталось допустимых меток, удаляем пару файлов
    if not is_bbox_valid:
        os.remove(img_fname2)
        os.remove(txt_fname2)

# Функция для замены номеров классов в метках
def replace_class_ids_in_labels(label_dir, matches_file):
    # Чтение соответствия классов из файла
    class_matches = {}
    with open(matches_file, 'r') as file:
        for line in file:
            parts = line.strip().split('|')
            if len(parts) == 2:
                old_class_id, new_class_id = map(int, parts)
                class_matches[old_class_id] = new_class_id

    # Обработка файлов меток
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    for label_file in tqdm(label_files, desc="Замена классов в метках", unit="file"):
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as file:
            lines = file.readlines()

        with open(label_path, 'w') as file:
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 1:
                    class_id = int(parts[0])
                    if class_id in class_matches:
                        parts[0] = str(class_matches[class_id])
                    file.write(' '.join(parts) + '\n')

# Функция для распределения файлов по папкам
def distribute_files(src, dst_dirs, proportions):
    # Получаем список всех файлов в исходной директории и разделяем на пары изображений и меток
    img_files = [f for f in os.listdir(src) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'))]
    # Создаем список пар изображений и соответствующих файлов меток
    pairs = [(img, img.rsplit('.', 1)[0] + '.txt') for img in img_files if os.path.exists(os.path.join(src, img.rsplit('.', 1)[0] + '.txt'))]

    # Распределяем пары файлов согласно заданным пропорциям
    total_pairs = len(pairs)
    distribution = [round(prop * total_pairs) for prop in proportions]

    # Перемешиваем пары дважды
    random.shuffle(pairs)
    random.shuffle(pairs)
    start = 0
    for dst, count in zip(dst_dirs, distribution):
        end = start + count
        selected_pairs = pairs[start:end]
        for img_file, txt_file in tqdm(selected_pairs, desc=f"Копирование в {dst}", leave=True):
            img_src_path = os.path.join(src, img_file)
            txt_src_path = os.path.join(src, txt_file)
            img_dst_path = os.path.join(dst, 'images', img_file)
            txt_dst_path = os.path.join(dst, 'labels', txt_file)
            shutil.copy(img_src_path, img_dst_path)
            shutil.copy(txt_src_path, txt_dst_path)
        start = end
        # Подсчет и вывод количества пар в каждом наборе
        num_pairs_img = len(os.listdir(os.path.join(dst, 'images')))
        num_pairs_lbs = len(os.listdir(os.path.join(dst, 'labels')))
        num_pairs = (num_pairs_img + num_pairs_lbs) // 2
        print(f"В папку {os.path.basename(dst)} распределено {num_pairs} пар.")

# Функция для рисования рамок на изображении
def draw_bounding_boxes(img_dir, classes_file):
    # Считываем названия классов
    with open(classes_file, 'r', encoding='utf-8') as f:
        classes = [line.strip() for line in f]

    # Определяем список цветов для каждого класса
    color_lst = [
        (36, 28, 237),   # красный
        (39, 127, 255),  # оранжевый
        (0, 242, 255),   # желтый
        (76, 177, 34),   # зеленый
        (232, 162, 0),   # голубой
        (204, 72, 63),   # синий
        (164, 73, 163),  # фиолетовый
        (21, 0, 136),    # коричневый
        (127, 127, 127), # серый
        (0, 0, 0)        # черный
    ]

    # Обрабатываем каждый файл изображения
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'))]
    for img_file in tqdm(img_files, desc="Рисование рамок", leave=True):
        img_path = os.path.join(img_dir, img_file)
        txt_path = os.path.splitext(img_path)[0] + '.txt'

        # Если файла меток нет, пропускаем изображение
        if not os.path.exists(txt_path):
            continue

        image = cv2.imread(img_path)
        with open(txt_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                class_id = int(float(parts[0]))
                x_center, y_center, width, height = map(float, parts[1:])
                x_min = int((x_center - width / 2) * image.shape[1])
                y_min = int((y_center - height / 2) * image.shape[0])
                x_max = int((x_center + width / 2) * image.shape[1])
                y_max = int((y_center + height / 2) * image.shape[0])

                # Выбираем цвет рамки в зависимости от класса
                bbox_color = color_lst[class_id % len(color_lst)]

                # Рисуем рамку и подпись класса на картинке
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), bbox_color, 1)
                if class_id < len(classes):
                    cv2.putText(image, classes[class_id], (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_color, 1)

        # Сохраняем измененное изображение и удаляем файл меток
        cv2.imwrite(img_path, image)
        os.remove(txt_path)

# Функция для создания yaml-файла структуры датасета yolo8
def create_yaml_file(dst_dir: str, class_lst):
  # Создание пути и имени файла конфигурации
  data_yaml_fname = os.path.join(dst_dir, 'data.yaml')
  try:
    # Открытие файла для записи конфигурации
    with open(data_yaml_fname, 'w', encoding='utf-8') as file_yaml:
      # Запись путей к наборам данных для тренировки, валидации и тестирования
      file_yaml.write('train: ../train/images\n')
      file_yaml.write('val: ../valid/images\n')
      file_yaml.write('test: ../test/images\n\n')
      # Запись количества классов
      file_yaml.write(f'nc: {len(class_lst)}\n\n')
      # Запись списка классов
      file_yaml.write(f'names: {class_lst}')
  except Exception as e:
    print(f'[ERROR] Error writing the list to a file: {e}')

# Главная исполняемая функция, в которую собрана вся логика по формированию рабочего датасета для YOLO8
def main_process(src_dir, dst_dir, classes_file, img_size):
    print("Начало обработки данных...")
    # Создание временной директории для обработки данных
    temp_dir = os.path.join(src_dir, 'Temp')
    # Создание директорий для тренировочных, валидационных и тестовых данных
    train_dir = os.path.join(dst_dir, 'train')
    valid_dir = os.path.join(dst_dir, 'valid')
    test_dir = os.path.join(dst_dir, 'test')

    # Создание необходимых директорий и поддиректорий
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(dst_dir, exist_ok=True)
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    for dir in [train_dir, valid_dir, test_dir]:
        os.makedirs(os.path.join(dir, 'images'), exist_ok=True)
        os.makedirs(os.path.join(dir, 'labels'), exist_ok=True)
    print("Директории созданы.")

    # Находим все пары изображение-метка
    print("Поиск пар изображение-метка...")
    pairs = find_image_label_pairs(src_dir)
    print(f"Найдено {len(pairs)} пар.\n")

    # Копируем и изменяем размер изображений
    print("Обработка и копирование изображений во временную директорию...")
    for img_path, txt_path in tqdm(pairs, desc="Обрабатываем изображения"):
        base_name = os.path.basename(img_path)
        new_img_path = os.path.join(temp_dir, base_name)
        new_txt_path = os.path.join(temp_dir, os.path.basename(txt_path))
        pair_normalize_copy(img_path, txt_path, new_img_path, new_txt_path, img_size)
    print("Изображения обработаны и скопированы.\n")

    # Заменяем номера классов в файлах меток на соответствующие нашему перечню из classes.txt
    print("Замена номеров меток на актуальные...")
    replace_class_ids_in_labels(temp_dir, matches_file)
    print("Метки приведены в соответствие.\n")

    # Распределяем файлы по папкам train, valid и test
    print("Распределение файлов по папкам train, valid и test...")
    distribute_files(temp_dir, [train_dir, valid_dir, test_dir], distribute_dataset)
    print("Файлы распределены.\n")

    # Рисуем рамки вокруг объектов
    print("Рисование рамок на изображениях в папке Temp...")
    draw_bounding_boxes(temp_dir, classes_file)
    print("\nРамки нарисованы.\n")

    # Создаем yaml файл для YOLOv8
    print("Создание файла конфигурации data.yaml...")
    class_lst = read_classes(classes_file)
    create_yaml_file(dst_dir, class_lst)
    print("Файл data.yaml создан.")

    print("Обработка данных завершена.")

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Путь к набору исходных датасетов, собранных с первичной подготовкой (есть файлы изображений и соответствующие им файлы меток).
# Эти датасеты могут располагаться в произвольном порядке и вложениями.
# Файлы меток исходно могут иметь не обязательно верную нумерацию классов. В этом случае надо заполнить файл class_matches.txt с указанием какой номер класса на какой заменить.
src_dir = '/content/drive/MyDrive/UII/Datasets'

# Путь к итоговому сводному датасету, который далее пойдет на обучение модели YOLO8
dst_dir = '/content/drive/MyDrive/UII/Dataset'

# Файл классов (внутри не менять, только указать корректный путь где лежит)
classes_file = '/content/drive/MyDrive/UII/classes.txt'

# Файл замен классов, которые надо провести при обработке первичных загруженных датасетов
matches_file = '/content/drive/MyDrive/UII/class_matches.txt'

# Рабочий размер квадрата изображения для передачи потом в YOLO8 (640 не менять)
img_size = 640

# Переключатель сохранять ли пропорции исходного изображения при ресайзе или сжать/растянуть в квадрат yolo
keep_proportions = True

# Доли сплитования наборов для обучения
distribute_dataset = [0.7, 0.2, 0.1]

# Запуск основного процесса обработки данных
main_process(src_dir, dst_dir, classes_file, img_size)

Начало обработки данных...
Директории созданы.
Поиск пар изображение-метка...
Найдено 623 пар.

Обработка и копирование изображений во временную директорию...


Обрабатываем изображения: 100%|██████████| 623/623 [05:28<00:00,  1.90it/s]


Изображения обработаны и скопированы.

Замена номеров меток на актуальные...


Замена классов в метках: 100%|██████████| 623/623 [00:19<00:00, 32.27file/s]


Метки приведены в соответствие.

Распределение файлов по папкам train, valid и test...


...

В папку valid распределено 125 пар.


Копирование в /content/drive/MyDrive/UII/Dataset/test: 100%|██████████| 62/62 [00:01<00:00, 33.76it/s]


В папку test распределено 62 пар.
Файлы распределены.

Рисование рамок на изображениях в папке Temp...


Рисование рамок: 100%|██████████| 623/623 [00:27<00:00, 22.49it/s]



Рамки нарисованы.

Создание файла конфигурации data.yaml...
Файл data.yaml создан.
Обработка данных завершена.


In [ ]:
# Удаление временной папки после окончания работы
# !!! Сперва дождитесь появления всех файлов в целевых наборах
# и выборочно визуально просмотрите корректность разметки на картинках в папке Temp
# надо подождать несколько минут для завершения этого процесса на Google Drive
temp_dir = os.path.join(src_dir, 'Temp')
delete_directory(temp_dir)